### Percobaan 1: Matching untuk 3 Driver dan Customer
#### Distance & Time(dgn Openrouteservice), Order Count & Rating.
*) Referensi code dari project semester lalu.

In [34]:
import pandas as pd
import pyomo.environ as pyo
import openrouteservice as ors
from dotenv import load_dotenv
import os

In [35]:
# Melakukan loading data client dan driver dari CSV.
# File dummy dari project semester lalu.

clients = pd.read_csv('../datasets/data_client.csv')
drivers = pd.read_csv('../datasets/data_driver.csv')

clients.head(3), drivers.head(3)

(   id  latitude   longitude
 0   0 -7.251299  112.789335
 1   1 -7.247283  112.756581
 2   2 -7.344951  112.760688,
    id  latitude   longitude  rating  order_count  total_trip
 0   0 -7.293717  112.747264       1          180    1571.400
 1   1 -7.360642  112.716350       0           71     143.562
 2   2 -7.257948  112.731777       3           83     489.202)

In [36]:
# Load top 3 data pertama menjadi list

clients_data = clients[:3].to_dict('records')
drivers_data = drivers[:3].to_dict('records')
clients_data, drivers_data

([{'id': 0, 'latitude': -7.251298999984013, 'longitude': 112.78933499965396},
  {'id': 1, 'latitude': -7.247282999983452, 'longitude': 112.75658099973666},
  {'id': 2, 'latitude': -7.344950999997104, 'longitude': 112.76068799972629}],
 [{'id': 0,
   'latitude': -7.293716999989942,
   'longitude': 112.74726399976018,
   'rating': 1,
   'order_count': 180,
   'total_trip': 1571.3999999998664},
  {'id': 1,
   'latitude': -7.360641999999297,
   'longitude': 112.71634999983824,
   'rating': 0,
   'order_count': 71,
   'total_trip': 143.56199999999984},
  {'id': 2,
   'latitude': -7.2579479999849426,
   'longitude': 112.73177699979928,
   'rating': 3,
   'order_count': 83,
   'total_trip': 489.2019999999644}])

In [37]:
# Definisikan weight.
weights = {
    'distance': { 'target': 10000, 'positive': 1, 'negative': -1 },
    'duration': { 'target': 5, 'positive': 1, 'negative': -1 },
    'order_count': { 'target': 20, 'positive': 1, 'negative': -1 },
    'rating': { 'target': 3, 'positive': -1, 'negative': 1 },
}

# Simpan key dari ORS
load_dotenv(override=True)
key = os.getenv('OPENROUTESERVICE_KEY')
client = ors.Client(key=key, base_url='https://api.openrouteservice.org/')

memo = {}

In [38]:
# Buat mapping utk distance, duration, order count, dan rating dari masing-masing pasangan customer dan driver.

def get_distance_and_duration(lat1, lon1, lat2, lon2): 
    routes = client.directions(((lon1, lat1), (lon2, lat2)), profile='driving-car')
    return {
            "distance": routes.get('routes')[0]['summary']['distance'], 
            "duration": routes.get('routes')[0]['summary']['duration']
        }

data: dict[tuple, dict[str, any]] = {}
for d in drivers_data:
    d_id = d['id']
    data[d_id] = {}
    for c in clients_data:
        c_id = c['id']
        result = memo.get((d['latitude'], d['longitude'], c['latitude'], c['longitude']))
        if result is None:
            result = get_distance_and_duration(d['latitude'], d['longitude'], c['latitude'], c['longitude'])
            memo[(d['latitude'], d['longitude'], c['latitude'], c['longitude'])] = result

        data[d_id][c_id] = {
            "distance": result['distance'],
            "duration": result['duration'],
            "order_count": d['order_count'],
            "rating": d['rating'],
        }

data

{0: {0: {'distance': 11244.2,
   'duration': 733.5,
   'order_count': 180,
   'rating': 1},
  1: {'distance': 6663.5, 'duration': 492.9, 'order_count': 180, 'rating': 1},
  2: {'distance': 11275.3,
   'duration': 893.4,
   'order_count': 180,
   'rating': 1}},
 1: {0: {'distance': 20932.6,
   'duration': 1289.7,
   'order_count': 71,
   'rating': 0},
  1: {'distance': 16351.9, 'duration': 1049.0, 'order_count': 71, 'rating': 0},
  2: {'distance': 6632.0, 'duration': 843.4, 'order_count': 71, 'rating': 0}},
 2: {0: {'distance': 8253.3,
   'duration': 611.5,
   'order_count': 83,
   'rating': 3},
  1: {'distance': 3672.6, 'duration': 370.9, 'order_count': 83, 'rating': 3},
  2: {'distance': 15714.5, 'duration': 997.5, 'order_count': 83, 'rating': 3}}}

In [39]:
# Model pyomo utk solve GP
model = pyo.ConcreteModel()
model.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)

# Define Variables
drivers = [d['id'] for d in drivers_data]
clients = [c['id'] for c in clients_data]

model.x = pyo.Var(drivers, clients, domain=pyo.NonNegativeIntegers)

# Define Objective
def get_penalty(target, val, positive=True):
  if positive:
    if val > target: return val - target
    else: return 0
  else:
    if val < target: return target - val
    else: return 0

# Penalty = (weight positive * X * deviasi dgn target / 1% dari target)
def penalty(c, d, param):
  m = weights[param]
  return (
    (m['positive'] * model.x[d, c] * get_penalty(m['target'], data[d][c][param]) / m['target'] / 100)+ 
    (m['negative'] * model.x[d, c] * get_penalty(m['target'], data[d][c][param], False) / m['target'] / 100)
  )

# Objective fn = Sum of penalty semua bidang.
model.Cost = pyo.Objective(
    expr = sum([
        penalty(c, d, 'distance') + 
        penalty(c, d, 'duration') + 
        penalty(c, d, 'order_count') + 
        penalty(c, d, 'rating') 
        for c in clients for d in drivers
      ]),
    sense = pyo.minimize)

# Define Constraint
# Constraint = jumlah dari setiap Driver dengan semua pasangan Clientnya adalah 1
model.driver = pyo.ConstraintList()
for d in drivers:
  model.driver.add(sum(model.x[d, c] for c in clients) == 1)

model.client = pyo.ConstraintList()
for c in clients:
  model.client.add(sum(model.x[d, c] for d in drivers) == 1)

In [40]:
# Gunakan solver utk solve
result = pyo.SolverFactory('cbc').solve(model)
result.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 4.01081547
  Upper bound: 4.01081547
  Number of objectives: 1
  Number of constraints: 6
  Number of variables: 9
  Number of nonzeros: 9
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.01
  Wallclock time: 0.01
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of cr

In [1]:
print("Hasil dari Assignment Matching\n")
result_matrix = []
ind = 0
for d in drivers:
  result_matrix.append([])
  for c in clients:
    result_matrix[ind].append(model.x[d,c]())
  ind += 1

pd.DataFrame(result_matrix)

Hasil dari Assignment Matching



NameError: name 'drivers' is not defined

In [42]:
from pprint import pprint

for i in range(len(result_matrix)):
    for j in range(len(result_matrix[i])):
        if result_matrix[i][j] == 1:
            print(f"Driver ID {i} -> Customer ID {j}")
            pprint(data[i])

Driver ID 0 -> Customer ID 0
{0: {'distance': 11244.2, 'duration': 733.5, 'order_count': 180, 'rating': 1},
 1: {'distance': 6663.5, 'duration': 492.9, 'order_count': 180, 'rating': 1},
 2: {'distance': 11275.3, 'duration': 893.4, 'order_count': 180, 'rating': 1}}
Driver ID 1 -> Customer ID 2
{0: {'distance': 20932.6, 'duration': 1289.7, 'order_count': 71, 'rating': 0},
 1: {'distance': 16351.9, 'duration': 1049.0, 'order_count': 71, 'rating': 0},
 2: {'distance': 6632.0, 'duration': 843.4, 'order_count': 71, 'rating': 0}}
Driver ID 2 -> Customer ID 1
{0: {'distance': 8253.3, 'duration': 611.5, 'order_count': 83, 'rating': 3},
 1: {'distance': 3672.6, 'duration': 370.9, 'order_count': 83, 'rating': 3},
 2: {'distance': 15714.5, 'duration': 997.5, 'order_count': 83, 'rating': 3}}
